In [1]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

2024-07-28 22:24:34.686203: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-28 22:24:34.824141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 22:24:34.884462: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 22:24:34.901912: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 22:24:35.002126: I tensorflow/core/platform/cpu_feature_guar

Default GPU Device: /device:GPU:0


I0000 00:00:1722185677.296357   12221 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722185677.427691   12221 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722185677.431587   12221 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722185677.589771   12221 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [2]:
import os
from PIL import Image

def get_image_dimensions(image_path):
    """
    Opens an image file and returns its dimensions.
    
    :param image_path: Path to the image file
    :return: A tuple containing the width and height of the image
    """
    try:
        with Image.open(image_path) as img:
            return img.size
    except IOError:
        print(f"Error opening {image_path}")
        return None

def process_images_in_folder(folder_path):
    """
    Goes through all files in a folder, opens image files, and prints their dimensions.
    
    :param folder_path: Path to the folder containing images
    """
    # List all files in the folder
    for filename in os.listdir(folder_path):
        # Construct full file path
        image_path = os.path.join(folder_path, filename)
        
        # Check if the file is an image by trying to open it with PIL.Image.open
        dimensions = get_image_dimensions(image_path)
        if dimensions:
            print(f"{filename}: {dimensions}")

# Example usage
folder_path = "../dataset/Cas16/liver"
process_images_in_folder(folder_path)

FileNotFoundError: [Errno 2] No such file or directory: '../dataset/Cas16/liver'

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

# TPU Setup
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("Running on TPU:", resolver.cluster_spec().as_dict()["worker"])
except:
    strategy = tf.distribute.get_strategy()
print(f"Running on {strategy.num_replicas_in_sync} replicas")

IMAGE_SIZE = (252, 252)
BATCH_SIZE = 16 #* strategy.num_replicas_in_sync

def preprocess_image(image_path, liver_mask_path, tumor_mask_path):
    def read_image(path):
        file = tf.io.read_file(path)
        image = tf.image.decode_jpeg(file, channels=1)
        image = tf.image.resize(image, (256, 256))  # Resize to nearest multiple of 3
        image = tf.image.resize(image, IMAGE_SIZE)  # Resize to exact size
        image = tf.cast(image, tf.float32) / 252.0
        return image

    img = read_image(image_path)
    liver_mask = read_image(liver_mask_path)
    tumor_mask = read_image(tumor_mask_path)

    combined_mask = tf.stack([liver_mask, tumor_mask], axis=-1)
    combined_mask = tf.squeeze(combined_mask, axis=2)  # Remove the extra dimension
    return img, combined_mask

def create_dataset(data_dir, case_ids):
    image_paths = []
    liver_mask_paths = []
    tumor_mask_paths = []

    for case_id in case_ids:
        case_path = os.path.join(data_dir, case_id)
        for file in os.listdir(os.path.join(case_path, 'images')):
            image_paths.append(os.path.join(case_path, 'images', file))
        for file in os.listdir(os.path.join(case_path, 'liver')):
            liver_mask_paths.append(os.path.join(case_path, 'liver', file))
        for file in os.listdir(os.path.join(case_path, 'tumor')):
            tumor_mask_paths.append(os.path.join(case_path, 'tumor', file))

    dataset = tf.data.Dataset.from_tensor_slices((
        image_paths,
        liver_mask_paths,
        tumor_mask_paths
    ))
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.cache()
    dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Load the dataset
all_case_ids = os.listdir('./dataset')
train_case_ids = all_case_ids[:int(0.8 * len(all_case_ids))]
val_case_ids = all_case_ids[int(0.8 * len(all_case_ids)):]

train_dataset = create_dataset('dataset', train_case_ids)
val_dataset = create_dataset('dataset', val_case_ids)

# Model definition and training
inputs = tf.keras.layers.Input(shape=(*IMAGE_SIZE, 1))
x = tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D()(x)
x = tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D()(x)
outputs = tf.keras.layers.Conv2D(2, 3, activation='sigmoid', padding='same')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    verbose=1
)

print("Training completed.")

Running on 1 replicas


I0000 00:00:1722185699.741038   12221 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722185699.744730   12221 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722185699.747777   12221 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722185699.751179   12221 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/50


I0000 00:00:1722185702.369693   12569 service.cc:146] XLA service 0x7795d4004e90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722185702.369717   12569 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-07-28 22:25:02.453525: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-28 22:25:02.742396: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
E0000 00:00:1722185706.642394   12569 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1722185706.796790   12569 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


 1/87 ━━━━━━━━━━━━━━━━━━━━ 18:08 13s/step - accuracy: 0.7750 - loss: 0.6933

...

86/87 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.9352 - loss: 0.3382

E0000 00:00:1722185722.589848   12570 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1722185722.739952   12570 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1722185723.272052   12570 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1722185723.422182   12570 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1722185723.577263   12570 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.9358 - loss: 0.3368

2024-07-28 22:25:24.901787: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion', 24 bytes spill stores, 36 bytes spill loads



87/87 ━━━━━━━━━━━━━━━━━━━━ 30s 206ms/step - accuracy: 0.9363 - loss: 0.3353 - val_accuracy: 0.9998 - val_loss: 0.1354
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 0.9455 - loss: 0.1333 - val_accuracy: 0.9998 - val_loss: 0.1428
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 10s 112ms/step - accuracy: 0.9915 - loss: 0.1059 - val_accuracy: 0.9995 - val_loss: 0.1415
Epoch 4/50
67/87 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - accuracy: 0.9989 - loss: 0.1058

In [28]:
model.save("final_model.h5")